### To create our Own dataset

In [2]:
import cv2
import numpy as np
import os

In [12]:
image_x, image_y = 50, 50

In [13]:
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

In [14]:
def store_images(g_id):
    total_pics=1200
    cap=cv2.VideoCapture(0)
    x,y,w,h=300,50,350,350
    create_folder("gestures/"+str(g_id))
    
    pic_no=0
    flag_start_capturing=False
    frames=0
    
    while True:
        ret,frame=cap.read()
        frame=cv2.flip(frame,1)
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask2=cv2.inRange(hsv,np.array([2,50,60]),np.array([25,150,255]))
        res=cv2.bitwise_and(frame,frame,mask=mask2)
        gray=cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
        median=cv2.GaussianBlur(gray,(5,5),0)
        kernel_square=np.ones((5,5),np.uint8)
        dilation=cv2.dilate(median,kernel_square,iterations=2)
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)
        ret,thresh=cv2.threshold(opening,30,255,cv2.THRESH_BINARY)
        thresh=thresh[y:y+h,x:x+w]
        contours=cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1]
        
        if len(contours)>0:
            contour=max(contours,key=cv2.contourArea)
            if cv2.contourArea(contour)>10000 and frames>50:
                x1,y1,w1,h1=cv2.boundingRect(contour)
                pic_no+=1
                save_img=thresh[y1:y1+h1,x1:x1+w1]
                save_img=cv2.resize(save_img,(image_x,image_y))
                cv2.putText(frame,"Capturing...",(30,60),cv2.FONT_HERSHEY_TRIPLEX,2,(127,255,255))
                cv2.imwrite("gestures/"+str(g_id)+"/"+str(pic_no)+".jpg",save_img)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(frame,str(pic_no),(30,400),cv2.FONT_HERSHEY_TRIPLEX,1.5,(127,127,255))
            cv2.imshow("Capturing gesture",frame)
            cv2.imshow("thresh",thresh)
            keypress=cv2.waitKey(1)
            if keypress==ord("c"):
                if flag_start_capturing==False:
                    flag_start_capturing=True
                else:
                    flag_start_capturing=False
                    frames=0
            if flag_start_capturing==True:
                frames+=1
            if pic_no==total_pics:
                break
                

In [ ]:
g_id=input("Enter gesture no:")
store_images(g_id)

Enter gesture no:2


In [15]:
cap=cv2.VideoCapture(0)
cap.release()
cv2.destroyAllWindows()

In [16]:
from PIL import Image
import numpy as np
import sys
import os
import csv
j=0

# default format can be changed as needed
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=True):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList

# load the original image

mydir=["hand-emo/1","hand-emo/2","hand-emo/3","hand-emo/4","hand-emo/5","hand-emo/6",
       "hand-emo/7","hand-emo/8","hand-emo/9","hand-emo/10","hand-emo/11","hand-emo/12"]

for i in mydir:
    j=j+1
    myFileList=createFileList(i)
    for file in myFileList:
        img_file = Image.open(file)
        #img_file.show()

        # get original image parameters...
        width, height = img_file.size
        format = img_file.format
        mode = img_file.mode

        # Make image Greyscale
        img_grey = img_file.convert('L')
        #img_grey.save('result.png')
        #img_grey.show()
    
        # Save Greyscale values
        value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((img_grey.size[1], img_grey.size[0]))
        for m in range(1200):
            value=np.append(value,j)
            value = value.flatten()
        with open("train_foo1.csv", 'a') as f:
            writer = csv.writer(f)
            writer.writerow(value)

hand-emo/1
hand-emo/2
hand-emo/3
hand-emo/4
hand-emo/5
hand-emo/6
hand-emo/7
hand-emo/8
hand-emo/9
hand-emo/10
hand-emo/11
hand-emo/12


In [5]:
import numpy as np
from keras.layers import Dense,Flatten,Conv2D
from keras.layers import MaxPooling2D,Dropout
from keras.utils import np_utils,print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd
import keras.backend as K

Using TensorFlow backend.


In [21]:
data = pd.read_csv("train_foo.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 0:2500]
Y = Y[:, 0]

In [22]:
print(X)
print(Y)

[['es/3' 0 0 ... 255 255 255]
 ['es/6' 0 0 ... 254 254 254]
 ['es/7' 0 0 ... 255 255 255]
 ...
 ['es/7' 0 0 ... 255 255 255]
 ['es/7' 0 0 ... 254 254 254]
 ['es/7' 0 0 ... 255 255 255]]
['es/3' 'es/6' 'es/7' ... 'es/7' 'es/7' 'es/7']


In [23]:
X_train = X[0:12000, :]
X_train = X_train/255
X_test = X[12000:13201, :]
X_test = X_test/255

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [10]:
Y=Y.reshape(Y.shape[0],1)
Y_train=Y[0:10800,:]
Y_train=Y_train.T
Y_test=Y[10800:12001,:]
Y_test=Y_test.T
print("number of training examples=" + str(X_train.shape[0]))
print("number of test examples:" + str(X_test.shape[0]))
print("X_train shape:" + str(X_train.shape))
print("Y_train shape:" + str(Y_train.shape))
print("X_test shape:" + str(X_test.shape))
print("Y_test shape:" + str(Y_test.shape))

number of training examples=9599


NameError: name 'X_test' is not defined

In [7]:
image_x=50
image_y=50
train_y=np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0],50,50,1)
X_test=X_test.reshape(X_test.shape[0],50,50,1)
print("X_train shape:"+str(X_train.shape))
print("X_test shape:"+str(X_test.shape))
print("train_y shape:"+str(train_y.shape))

X_train shape:(10800, 50, 50, 1)
X_test shape:(1201, 50, 50, 1)
train_y shape:(10800, 13)


In [24]:
def keras_model(image_x,image_y):
    num_of_classes=13
    model=Sequential()
    model.add(Conv2D(32,(5,5),input_shape=(image_x,image_y,1),activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="same"))
    model.add(Conv2D(64,(5,5),activation="sigmoid"))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding="same"))
    model.add(Flatten())
    model.add(Dense(1024,activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes,activation="softmax"))
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    filepath="handEmo.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor="val_acc",verbose=1,save_best_only=True,mode="max")
    callbacks_list=[checkpoint1]
    return model,callbacks_list

In [25]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(X_train,train_y,validation_data=(X_test,test_y),epochs=1,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(X_test,test_y,verbose=0)

print("CNN Error:%.2f%%"%(100-scores[1]*100))
print_summary(model)
model.save("handEmo.h5")

NameError: name 'train_y' is not defined

In [80]:
from keras.models import load_model

In [81]:
model=load_model("handEmo.h5")

In [82]:
def get_emojis():
    emoji_folder="hand-emo/"
    emojis=[]
    for emoji in range(len(os.listdir(emoji_folder))):
        print(emoji)
        emojis.append(cv2.imread(emoji_folder+str(emoji)+".jpg",-1))
    return emojis

In [83]:
emojis=get_emojis()
cap=cv2.VideoCapture(0)
x,y,w,h=300,50,350,350

0
1
2
3
4
5
6
7
8
9
10
11


In [84]:
def keras_predict(model,image):
    processed=keras_process_image(image)
    pred_probab=model.predict(processed)[0]
    pred_class=list(pred_probab).index(max(pred_probab))
    return max(pred_probab),pred_class

In [85]:

def keras_process_image(img):
    image_x=50
    image_y=50
    img=cv2.resize(img,(image_x,image_y))
    img=np.array(img,dtype=np.float32)
    img=np.reshape(img,(-1,image_x,image_y,1))
    return img

In [86]:
def overlay(image,emoji,x,y,w,h):
    emoji=cv2.resize(emoji,(w,h))
    try:
        image[y:y+h,x:x+w]=blend_transparent(image[y:y+h,x:x+w],emoji)
    except:
        pass
    return image

In [87]:

def blend_transparent(face_img,overlay_t_img):
    overlay_img=overlay_t_img[:,:,:3]
    overlay_mask=overlay_t_img[:,:,3:]
    background_mask=255-overlay_mask
    overlay_mask=cv2.cvtColor(overlay_mask,cv2.COLOR_GRAY2BGR)
    background_mask=cv2.cvtColor(background_mask,cv2.COLOR_GRAY2BGR)
    face_part=(face_img*(1/255.0))*(backgroud_mask*(1/255.0))
    overlay_part=(overlay_img*(1/255.0))*(overlay_mask*(1/255.0))
    return np.uint8(cv.addWeighted(face_part,255.0,overlay_part,255.0,0.0))

In [88]:
while(cap.isOpened()):
    ret,img=cap.read()
    img=cv2.flip(img,1)
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask2=cv2.inRange(hsv,np.array([2,50,60]),np.array([25,150,255]))
    res=cv2.bitwise_and(img,img,mask=mask2)
    gray=cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    median=cv2.GaussianBlur(gray,(5,5),0)
    kernel_square=np.ones((5,5),np.uint8)
    dilation=cv2.dilate(median,kernel_square,iterations=2)
    opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)
    ret,thresh=cv2.threshold(opening,30,255,cv2.THRESH_BINARY)
    thresh=thresh[y:y+h,x:x+h]
    contours=cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1]
    if len(contours)>0:
        contour=max(contours,key=cv2.contourArea)
        if cv2.contourArea(contour)>2500:
            x,y,w1,h1=cv2.boundingRect(contour)
            newImage=thresh[y:y+h1,x:x+w1]
            newImage=cv2.resize(newImage,(50,50))
            pred_probab,pred_class=keras_predict(model,newImage)
            print(pred_class,pred_probab)
            img=overlay(img,emojis[pred_class],400,250,90,90)
    x,y,w,h=300,50,350,350
    cv2.imshow("Frame",img)
    cv2.imshow("Contours",thresh)
    k=cv2.waitKey(10)
    if k==27:
        break
keras_predict(model,np.zeros((50,50,1),dtype=np.uint8))


3 0.7411356
2 0.9035153
4 0.9996438
3 0.7248678
4 0.6174232
8 0.54916096
8 0.76728773
8 0.82766026
8 0.9232102
8 0.9560775
8 0.9302352
8 0.91253686
8 0.93498504
8 0.7665699
8 0.68729585
7 0.33277714
7 0.55633664
7 0.5665143
7 0.67538476
5 0.44709632
8 0.35831296
8 0.497903
11 0.7193326
11 0.3915754
1 0.86221015
1 0.9407709
1 0.60810524
11 0.49728215
1 0.8964761
1 0.9355114
1 0.8859514
1 0.83983
1 0.9030553
1 0.8880984
1 0.73888177
1 0.9067268
1 0.710809
1 0.761532
1 0.5877008
1 0.76888525
1 0.7731649
11 0.86473024
11 0.6084427
11 0.7435866
11 0.6531566
1 0.50014776
1 0.5394505
1 0.7757474
11 0.5322084
11 0.39461377
1 0.49998504
11 0.5738448
11 0.34227687
11 0.47989413
1 0.3598336
11 0.47930735
9 0.3296144
1 0.28544322
11 0.8386497
11 0.2603937
11 0.7722809
11 0.3798103
11 0.48313037
11 0.64389235
11 0.90797555
11 0.3493959
11 0.6661172
11 0.5653474
11 0.5078414
11 0.3847463
8 0.29815546
8 0.28701642
3 0.5214987
8 0.4563051
3 0.33696413
3 0.38913783
3 0.28551698
3 0.5076386
3 0.3446913


9 0.5828934
9 0.67483383
9 0.7992972
9 0.8554439
9 0.82499576
11 0.5094629
7 0.3694761
7 0.5716649
11 0.46931025
7 0.77493256
7 0.7608305
7 0.8313678
7 0.98095727
7 0.9925404
7 0.99433494
7 0.99876606
7 0.9993561
7 0.998052
7 0.99394673
7 0.99701947
7 0.9937947
7 0.9691046
7 0.8762129
7 0.6137604
7 0.5533694
7 0.6630942
7 0.6450699
7 0.49621558
7 0.41881412
1 0.4367786
7 0.22399111
1 0.6122298
1 0.66206384
2 0.32938716
2 0.45448762
2 0.54294133
2 0.62513274
2 0.71182823
2 0.7837503
2 0.7846632
2 0.65458786
2 0.70598876
4 0.58747816
4 0.5723392
4 0.4806022
8 0.95081335
8 0.9337448
8 0.97476983
8 0.9710764
8 0.9752603
8 0.98496616
8 0.9828987
8 0.9745256
8 0.96019477
8 0.98746455
8 0.9907364
8 0.9678715
8 0.99076086
8 0.9884085
8 0.99327224
8 0.99442947
8 0.9868586
8 0.98379636
8 0.9891123
8 0.97512656
8 0.94082636
12 0.64671296


IndexError: list index out of range

In [89]:
cap=cv2.VideoCapture(0)
cap.release()
cv2.destroyAllWindows()

In [176]:
cv2.imshow(emojis[1])

TypeError: Required argument 'mat' (pos 2) not found